In [52]:
import boto3
from botocore.vendored import requests
import twitter
import base64
session = boto3.session.Session(region_name='us-west-2')
rek = session.client('rekognition')
ddb = session.resource('dynamodb').Table('celebrity-handler')
ssm = session.client('ssm')
api = twitter.Api(*ssm.get_parameters(Names=[os.getenv("SSM_PARAMETER_NAME", "CFN-APIKeys-GMVD9X2JJsLO")])['Parameters'][0]['Value'].split(','))
TWITTER_SN = api.VerifyCredentials().screen_name

def validate_record(payload):
    if (
        TWITTER_SN in payload.get('text', '') and
        payload.get('entities', {}).get('media') and
        'RT' not in payload.get('text')
    ):
        return True
    return False

def process_record(payload):
    requests.get(payload['entities']['media'][0]['media_url']+":large")
    celebs = rek.recognize_celebrities(Image={'Bytes': resp.content})['CelebrityFaces']
    handles = []
    for celeb in celebs:
        ddb_resp = ddb.get_item(Key={"id": celeb['Id']})
        if ddb_resp.get('Item'):
            handles.append(ddb_resp['Item']['handle'])
    if not handles:
        api.PostUpdate(
            "There was no one famous in your picture! #lame",
            in_reply_to_status_id=payload['id']
        )
    status = "That looks like a picture of {}".format(', '.join(handles))
    api.PostUpdate(status, in_reply_to_status_id=payload['id'])
    

def lambda_handler(event, context):
    for record in event['Records']:
        payload = json.loads(base64.b64decode(record['kinesis']['data']))
        if not validate_record(payload):
            return
        process_record(payload)
    
    


In [53]:
resp

{u'CelebrityFaces': [{u'Face': {u'BoundingBox': {u'Height': 0.6118518710136414,
     u'Left': 0.273333340883255,
     u'Top': 0.2029629647731781,
     u'Width': 0.4588888883590698},
    u'Confidence': 99.99918365478516,
    u'Landmarks': [{u'Type': u'eyeLeft',
      u'X': 0.4332115650177002,
      u'Y': 0.45033934712409973},
     {u'Type': u'eyeRight',
      u'X': 0.5812413692474365,
      u'Y': 0.46330589056015015},
     {u'Type': u'nose', u'X': 0.5083878040313721, u'Y': 0.5419089794158936},
     {u'Type': u'mouthLeft',
      u'X': 0.43395864963531494,
      u'Y': 0.6721764206886292},
     {u'Type': u'mouthRight',
      u'X': 0.5558940768241882,
      u'Y': 0.6813442707061768}],
    u'Pose': {u'Pitch': 10.68148422241211,
     u'Roll': 3.123420476913452,
     u'Yaw': 3.5766777992248535},
    u'Quality': {u'Brightness': 36.461856842041016,
     u'Sharpness': 99.9980239868164}},
   u'Id': u'3CI7QV9d',
   u'MatchConfidence': 100.0,
   u'Name': u'Elon Musk',
   u'Urls': [u'www.imdb.com/nam